In [1]:
!pip install langchain langchain-openai langchain-community
!pip install faiss-cpu sentence-transformers
!pip install pypdf python-docx
!pip install tiktoken chromadb
!pip install llama-index
!pip install openai anthropic cohere
!pip install transformers torch
!pip install pinecone-client
import os
import sys
import warnings
import numpy as np
from typing import List, Dict, Any, Tuple
from dataclasses import dataclass
import pickle
from pathlib import Path

import faiss
import chromadb
from sentence_transformers import SentenceTransformer
import openai
from transformers import pipeline

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.schema import Document
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA


import tempfile
import torch
from IPython.display import display, Markdown

warnings.filterwarnings('ignore')


@dataclass
class RAGConfig:
    """Configuration class for the RAG system parameters."""
    chunk_size: int = 1000
    chunk_overlap: int = 200
    top_k_retrieval: int = 5
    embedding_model: str = "all-MiniLM-L6-v2"
    temperature: float = 0.1
    max_tokens: int = 500
    vector_store_type: str = "faiss"

class MedicalDocumentRAG:
    """
    A RAG system specialized for medical documents, supporting multiple
    vector stores (FAISS, ChromaDB, Pinecone) and LLMs (OpenAI, Anthropic, Cohere, Local HF).
    """

    def __init__(self, config: RAGConfig = None, api_keys: Dict[str, str] = None):
        self.config = config or RAGConfig()
        self.embedding_model = None
        self.vector_store = None
        self.documents = []
        self.text_chunks = []
        self.embeddings = None
        self.chunk_to_source = {}
        self.api_keys = api_keys or {}
        self.llm = None
        self.llm_type = "none"

        self._initialize_embedding_model()
        self._initialize_llm()

        self.prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a medical assistant. Answer the question based ONLY on the provided medical context.
Do not use any external knowledge. If the answer cannot be found in the context, say "I cannot answer this question based on the provided medical document."

Context:
{context}

Question: {question}

Answer: """
        )

    def _initialize_embedding_model(self):
        try:
            print(f"Loading embedding model: {self.config.embedding_model}")
            self.embedding_model_langchain = HuggingFaceEmbeddings(
                model_name=self.config.embedding_model,
                model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
            )
            self.embedding_model = SentenceTransformer(
                self.config.embedding_model,
                device='cuda' if torch.cuda.is_available() else 'cpu'
            )
            print("✓ Embedding model loaded successfully")
        except Exception as e:
            print(f"Error loading embedding model: {e}")
            raise

    def _initialize_llm(self):
        self.llm = None
        self.llm_type = "none"

        if "openai" in self.api_keys:
            try:
                os.environ["OPENAI_API_KEY"] = self.api_keys["openai"]
                self.llm = ChatOpenAI(
                    temperature=self.config.temperature,
                    max_tokens=self.config.max_tokens,
                    model_name="gpt-3.5-turbo"
                )
                self.llm_type = "openai"
                print("✓ OpenAI LLM initialized")
                return
            except Exception as e:
                print(f"OpenAI initialization failed: {e}")

        if "anthropic" in self.api_keys:
            try:
                from langchain_anthropic import ChatAnthropic
                self.llm = ChatAnthropic(
                    anthropic_api_key=self.api_keys["anthropic"],
                    temperature=self.config.temperature,
                    max_tokens=self.config.max_tokens
                )
                self.llm_type = "anthropic"
                print("✓ Anthropic LLM initialized")
                return
            except Exception as e:
                print(f"Anthropic initialization failed: {e}")

        if "cohere" in self.api_keys:
            try:
                from langchain_community.llms import Cohere
                self.llm = Cohere(
                    cohere_api_key=self.api_keys["cohere"],
                    temperature=self.config.temperature,
                    max_tokens=self.config.max_tokens
                )
                self.llm_type = "cohere"
                print("✓ Cohere LLM initialized")
                return
            except Exception as e:
                print(f"Cohere initialization failed: {e}")

        try:
            device = 0 if torch.cuda.is_available() else -1
            self.llm = pipeline("text-generation",
                              model="distilgpt2",
                              device=device,
                              max_length=512)
            self.llm_type = "local"
            print(f"✓ Using local Hugging Face model (distilgpt2) on device {device}")
        except Exception as e:
            print(f"Local model initialization failed: {e}")
            print("Warning: No LLM available. System will provide retrieval-only responses.")

    def upload_and_load_document(self) -> str | None:
        """Handles document upload (designed for Colab/Jupyter) and loading."""
        print("Please upload your medical document (PDF, TXT, or DOCX)...")

        # Colab specific upload
        uploaded = files.upload()

        if not uploaded:
            print("🚨 No file uploaded. Returning to main menu.")
            return None

        # Get the first uploaded file's name (assuming single file upload)
        filename = list(uploaded.keys())[0]

        # Save content to a temp file
        temp_dir = tempfile.gettempdir()
        temp_path = os.path.join(temp_dir, filename)

        with open(temp_path, 'wb') as f:
            f.write(uploaded[filename])

        print(f"File uploaded and saved temporarily as: {temp_path}")
        self.load_document(temp_path)
        return temp_path
    
    def load_document(self, file_path: str, file_extension: str = None) -> bool:
        try:
            if file_extension is None:
                file_extension = Path(file_path).suffix.lower()

            print(f"Loading document: {file_path}")

            if file_extension == '.pdf':
                loader = PyPDFLoader(file_path)
            elif file_extension == '.txt':
                loader = TextLoader(file_path, encoding='utf-8')
            elif file_extension in ['.docx', '.doc']:
                loader = Docx2txtLoader(file_path)
            else:
                raise ValueError(f"Unsupported file type: {file_extension}")

            self.documents = loader.load()
            print(f"✓ Document loaded successfully. Pages/chunks: {len(self.documents)}")

            return True

        except Exception as e:
            print(f"Error loading document: {e}")
            return False

    def process_and_chunk_document(self) -> bool:
        try:
            if not self.documents:
                print("No document loaded. Please load a document first.")
                return False

            print("Processing and chunking document...")

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=self.config.chunk_size,
                chunk_overlap=self.config.chunk_overlap,
                length_function=len,
                separators=["\n\n", "\n", " ", ""]
            )

            self.text_chunks = []
            self.chunk_to_source = {}

            for doc_idx, document in enumerate(self.documents):
                chunks = text_splitter.split_documents([document])

                for chunk_idx, chunk in enumerate(chunks):
                    global_chunk_idx = len(self.text_chunks)
                    self.text_chunks.append(chunk)

                    self.chunk_to_source[global_chunk_idx] = {
                        'document_index': doc_idx,
                        'chunk_index': chunk_idx,
                        'source': chunk.metadata.get('source', 'Unknown'),
                        'page': chunk.metadata.get('page', 'Unknown')
                    }

            print(f"✓ Document chunked into {len(self.text_chunks)} pieces")
            return True

        except Exception as e:
            print(f"Error processing document: {e}")
            return False

    def create_embeddings_and_index(self) -> bool:
        try:
            if not self.text_chunks:
                print("No text chunks available. Please process document first.")
                return False

            print("Creating embeddings for text chunks...")

            chunk_texts = [chunk.page_content for chunk in self.text_chunks]

            vs_type = self.config.vector_store_type.lower()

            if vs_type == "faiss":
                self._create_faiss_index(chunk_texts)
            elif vs_type == "chromadb":
                self._create_chromadb_index(chunk_texts)
            elif vs_type == "pinecone":
                self._create_pinecone_index(chunk_texts)
            else:
                print(f"Unsupported vector store type '{self.config.vector_store_type}', falling back to FAISS.")
                self._create_faiss_index(chunk_texts)

            return True

        except Exception as e:
            print(f"Error creating embeddings and index: {e}")
            return False

    def _create_faiss_index(self, chunk_texts: List[str]):
        self.embeddings = self.embedding_model.encode(
            chunk_texts,
            show_progress_bar=True,
            convert_to_numpy=True
        )

        dimension = self.embeddings.shape[1]
        self.vector_store = faiss.IndexFlatIP(dimension)

        faiss.normalize_L2(self.embeddings)
        self.vector_store.add(self.embeddings.astype(np.float32))

        print(f"✓ Created FAISS index with {self.vector_store.ntotal} vectors")
        print(f"✓ Embedding dimension: {dimension}")

    def _create_chromadb_index(self, chunk_texts: List[str]):
        client = chromadb.Client()
        collection = client.create_collection(name="medical_docs", embedding_function=None)

        embeddings = self.embedding_model.encode(chunk_texts, convert_to_numpy=True)

        for i, (text, embedding) in enumerate(zip(chunk_texts, embeddings)):
            collection.add(
                embeddings=[embedding.tolist()],
                documents=[text],
                ids=[str(i)]
            )

        self.vector_store = collection
        self.embeddings = embeddings
        print(f"✓ Created ChromaDB index with {len(chunk_texts)} vectors")

    def _create_pinecone_index(self, chunk_texts: List[str]):
        try:
            import pinecone

            if "pinecone" not in self.api_keys:
                raise ValueError("Pinecone API key required")

            index_name = "medical-rag-index"
            dimension = self.embedding_model.get_sentence_embedding_dimension()

            pinecone.init(api_key=self.api_keys["pinecone"], environment="us-west1-gcp") # Using a default region

            if index_name not in pinecone.list_indexes():
                pinecone.create_index(index_name, dimension=dimension)

            index = pinecone.Index(index_name)

            embeddings = self.embedding_model.encode(chunk_texts, convert_to_numpy=True)

            vectors = []
            for i, (text, embedding) in enumerate(zip(chunk_texts, embeddings)):
                vectors.append((str(i), embedding.tolist(), {"text": text}))

            batch_size = 100
            for i in range(0, len(vectors), batch_size):
                index.upsert(vectors[i:i+batch_size])

            self.vector_store = index
            self.embeddings = embeddings

            print(f"✓ Created Pinecone index with {len(chunk_texts)} vectors")

        except Exception as e:
            print(f"Pinecone setup failed, falling back to FAISS: {e}")
            self.config.vector_store_type = "faiss"
            self._create_faiss_index(chunk_texts)

    def retrieve_relevant_chunks(self, query: str, top_k: int = None) -> List[Tuple[Document, float, Dict]]:
        try:
            if self.vector_store is None:
                raise ValueError("Vector store not initialized. Please create embeddings first.")

            top_k = top_k or self.config.top_k_retrieval

            vs_type = self.config.vector_store_type.lower()

            if vs_type == "faiss" or hasattr(self.vector_store, 'search'):
                return self._retrieve_faiss(query, top_k)
            elif vs_type == "chromadb":
                return self._retrieve_chromadb(query, top_k)
            elif vs_type == "pinecone":
                return self._retrieve_pinecone(query, top_k)
            else:
                return self._retrieve_faiss(query, top_k)

        except Exception as e:
            print(f"Error retrieving chunks: {e}")
            return []

    def _retrieve_faiss(self, query: str, top_k: int) -> List[Tuple[Document, float, Dict]]:
        query_embedding = self.embedding_model.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(query_embedding)

        similarities, indices = self.vector_store.search(
            query_embedding.astype(np.float32),
            top_k
        )

        results = []
        for i, (similarity, idx) in enumerate(zip(similarities[0], indices[0])):
            if idx < len(self.text_chunks):
                chunk = self.text_chunks[idx]
                source_info = self.chunk_to_source.get(idx, {})
                results.append((chunk, float(similarity), source_info))

        return results

    def _retrieve_chromadb(self, query: str, top_k: int) -> List[Tuple[Document, float, Dict]]:
        query_embedding = self.embedding_model.encode([query], convert_to_numpy=True)

        results = self.vector_store.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k
        )

        retrieved = []
        for i, (doc_id, distance) in enumerate(zip(results['ids'][0], results['distances'][0])):
            idx = int(doc_id)
            if idx < len(self.text_chunks):
                chunk = self.text_chunks[idx]
                source_info = self.chunk_to_source.get(idx, {})
                similarity = 1 - distance
                retrieved.append((chunk, similarity, source_info))

        return retrieved

    def _retrieve_pinecone(self, query: str, top_k: int) -> List[Tuple[Document, float, Dict]]:
        query_embedding = self.embedding_model.encode([query], convert_to_numpy=True)

        results = self.vector_store.query(
            vector=query_embedding.tolist(),
            top_k=top_k,
            include_metadata=True
        )

        retrieved = []
        for match in results['matches']:
            idx = int(match['id'])
            if idx < len(self.text_chunks):
                chunk = self.text_chunks[idx]
                source_info = self.chunk_to_source.get(idx, {})
                similarity = match['score']
                retrieved.append((chunk, similarity, source_info))

        return retrieved

    def generate_answer(self, query: str, top_k: int = None) -> Dict[str, Any]:
        try:
            print(f"Processing query: {query}")

            retrieved_chunks = self.retrieve_relevant_chunks(query, top_k)

            if not retrieved_chunks:
                return {
                    'query': query,
                    'answer': 'I cannot answer this question based on the provided medical document.',
                    'sources': [],
                    'retrieved_chunks': 0
                }

            context_pieces = []
            sources = []

            for i, (chunk, similarity, source_info) in enumerate(retrieved_chunks):
                context_pieces.append(f"[Source {i+1}]: {chunk.page_content}")
                sources.append({
                    'source_id': i+1,
                    'similarity': similarity,
                    'content': chunk.page_content[:200] + '...' if len(chunk.page_content) > 200 else chunk.page_content,
                    'metadata': source_info
                })

            context = "\n\n".join(context_pieces)

            if self.llm is None or self.llm_type == "none":
                return {
                    'query': query,
                    'answer': f'Retrieved relevant context but no LLM available for generation. Here are the most relevant excerpts:\n\n{self._create_summary_from_context(context, query)}',
                    'sources': sources,
                    'retrieved_chunks': len(retrieved_chunks),
                    'context_length': len(context),
                    'llm_type': 'retrieval_only'
                }

            try:
                prompt = self.prompt_template.format(context=context, question=query)

                if self.llm_type in ["openai", "anthropic", "cohere"]:
                    answer = self.llm.invoke(prompt)
                    if hasattr(answer, 'content'):
                        answer = answer.content
                elif self.llm_type == "local":
                    max_gen_length = len(prompt.split()) + self.config.max_tokens
                    result = self.llm(prompt, max_length=max_gen_length,
                                    do_sample=True, temperature=0.7, pad_token_id=50256)
                    if isinstance(result, list) and len(result) > 0:
                        answer = result[0]['generated_text']
                        answer = answer[len(prompt):].strip()
                    else:
                        answer = "Unable to generate response with local model."
                else:
                    answer = str(self.llm.generate([prompt]))

                return {
                    'query': query,
                    'answer': answer.strip() if hasattr(answer, 'strip') else str(answer).strip(),
                    'sources': sources,
                    'retrieved_chunks': len(retrieved_chunks),
                    'context_length': len(context),
                    'llm_type': self.llm_type
                }

            except Exception as llm_error:
                error_msg = str(llm_error)

                if "quota" in error_msg.lower() or "429" in error_msg or "insufficient_quota" in error_msg:
                    llm_fail_type = 'quota_exceeded'
                elif "rate" in error_msg.lower() or "limit" in error_msg.lower():
                    llm_fail_type = 'rate_limited'
                else:
                    llm_fail_type = f'error_{self.llm_type}'

                return {
                    'query': query,
                    'answer': f'LLM failure ({llm_fail_type}). Here is the retrieved context:\n\n{self._create_summary_from_context(context, query)}',
                    'sources': sources,
                    'retrieved_chunks': len(retrieved_chunks),
                    'context_length': len(context),
                    'llm_type': llm_fail_type,
                    'error': error_msg
                }

        except Exception as e:
            print(f"Error generating answer: {e}")
            return {
                'error': str(e),
                'query': query,
                'answer': None,
                'sources': []
            }

    def _create_summary_from_context(self, context: str, query: str) -> str:

        """Create a simple rule-based summary when LLM is unavailable"""
        lines = context.split('\n')
        relevant_lines = []

        query_words = set(query.lower().split())

        for line in lines:
            if line.strip():
                line_words = set(line.lower().split())
                if query_words.intersection(line_words) or any(q_word in line.lower() for q_word in query_words):
                    relevant_lines.append(line.strip())

        if relevant_lines:
            summary = "Based on the medical document, here are the most relevant excerpts:\n\n"
            for i, line in enumerate(relevant_lines[:5], 1): # Show up to 5 relevant lines
                summary += f"*{i}. {line}*\n\n"
            return summary
        else:
            return context[:500] + "..." if len(context) > 500 else context

    def create_retrieval_qa_chain(self):
        """Creates a standard LangChain RetrievalQA chain using FAISS and self.llm."""
        try:
            if not self.llm:
                print("Cannot create QA chain: No LLM initialized.")
                return None

            if self.config.vector_store_type.lower() == "faiss":
                vectorstore = FAISS.from_documents(
                    self.text_chunks,
                    self.embedding_model_langchain
                ) # Fix: Dedent this closing parenthesis

                qa_chain = RetrievalQA.from_chain_type(
                    llm=self.llm,
                    chain_type="stuff",
                    retriever=vectorstore.as_retriever(search_kwargs={"k": self.config.top_k_retrieval}),
                    return_source_documents=True
                )

                print("✓ LangChain RetrievalQA Chain created successfully.")
                return qa_chain
            else:
                print(f"LangChain QA chain creation currently only supports FAISS, not {self.config.vector_store_type}.")
                return None

        except Exception as e:
            print(f"Error creating QA chain: {e}")
            return None

    def save_index(self, filepath: str = "/content/medical_rag_index.pkl"):
        try:
            data = {
                'embeddings': self.embeddings,
                'text_chunks': self.text_chunks,
                'chunk_to_source': self.chunk_to_source,
                'config': self.config
            }

            with open(filepath, 'wb') as f:
                pickle.dump(data, f)

            if self.config.vector_store_type.lower() == "faiss" and hasattr(self.vector_store, 'ntotal'):
                faiss_path = filepath.replace('.pkl', '_faiss.index')
                faiss.write_index(self.vector_store, faiss_path)

            print(f"✓ Index saved to {filepath}")

        except Exception as e:
            print(f"Error saving index: {e}")

    def load_index(self, filepath: str = "/content/medical_rag_index.pkl"):
        try:
            with open(filepath, 'rb') as f:
                data = pickle.load(f)

            self.embeddings = data['embeddings']
            self.text_chunks = data['text_chunks']
            self.chunk_to_source = data['chunk_to_source']
            self.config = data['config']

            if self.config.vector_store_type.lower() == "faiss":
                faiss_path = filepath.replace('.pkl', '_faiss.index')
                if os.path.exists(faiss_path):
                    self.vector_store = faiss.read_index(faiss_path)

            print(f"✓ Index loaded from {filepath}")
            return True

        except Exception as e:
            print(f"Error loading index: {e}")
            return False

    def get_system_stats(self) -> Dict[str, Any]:
        return {
            'documents_loaded': len(self.documents),
            'text_chunks': len(self.text_chunks),
            'embedding_model': self.config.embedding_model,
            'vector_store_type': self.config.vector_store_type,
            'chunk_size': self.config.chunk_size,
            'chunk_overlap': self.config.chunk_overlap,
            'top_k_retrieval': self.config.top_k_retrieval,
            'vector_store_initialized': self.vector_store is not None,
            'llm_initialized': self.llm is not None,
            'llm_type': self.llm_type
        }


def create_sample_document(filepath: str = "/content/sample_medical_doc.txt") -> str:
    """Creates a sample text file for quick testing."""
    sample_text = """
    ## Introduction to Chronic Fatigue Syndrome (CFS)
    Chronic Fatigue Syndrome (CFS), also known as Myalgic Encephalomyelitis (ME/CFS), is a complex,
    long-term illness that affects many body systems. It is characterized by severe fatigue that is
    not alleviated by rest and is worsened by physical or mental activity (post-exertional malaise).

    ## Diagnostic Criteria
    Diagnosis requires all three core symptoms:
    1. A substantial reduction or impairment in the ability to engage in pre-illness levels of
       occupational, educational, social, or personal activities, that persists for more than 6 months.
    2. Post-exertional malaise (PEM).
    3. Unrefreshing sleep.

    In addition, patients must also have at least one of the two following manifestations:
    * Cognitive impairment ("brain fog").
    * Orthostatic intolerance (problems with blood flow control).

    ## Treatment and Management
    There is no single cure for ME/CFS. Treatment focuses on symptom management and includes:
    * **Pacing:** Careful management of activity to avoid PEM.
    * **Medications:** For sleep, pain, and orthostatic intolerance.
    * **Cognitive Behavioral Therapy (CBT):** May help manage the impact of the illness, but is not a cure.
    """
    with open(filepath, 'w') as f:
        f.write(sample_text)
    return filepath

def get_api_keys_colab() -> Dict[str, str]:
    """Prompts for API keys using standard Python input for non-blocking flow."""
    api_keys = {}
    print("=" * 50)
    print("API Configuration (Optional: Use to enable powerful LLMs/Pinecone)")
    print("=" * 50)

    # Use environment variables if set, otherwise prompt (or default to empty)
    api_keys['openai'] = os.getenv("OPENAI_API_KEY", input("Enter OpenAI API key (or Enter): ").strip())
    api_keys['anthropic'] = os.getenv("ANTHROPIC_API_KEY", input("Enter Anthropic API key (or Enter): ").strip())
    api_keys['cohere'] = os.getenv("COHERE_API_KEY", input("Enter Cohere API key (or Enter): ").strip())
    api_keys['pinecone'] = os.getenv("PINECONE_API_KEY", input("Enter Pinecone API key (or Enter): ").strip())

    # Filter out empty keys
    return {k: v for k, v in api_keys.items() if v}

def demonstrate_rag_system():
    display(Markdown("## 💉 Medical Document RAG System"))

    api_keys = get_api_keys_colab()

    print("\n--- RAG Configuration ---")
    vector_store_choice = input("Choose vector store (faiss/chromadb/pinecone) [default: faiss]: ").strip().lower() or "faiss"

    if vector_store_choice == "pinecone" and 'pinecone' not in api_keys:
        print(" WARNING: Pinecone selected but API key is missing. Falling back to FAISS.")
        vector_store_choice = "faiss"

    config = RAGConfig(
        chunk_size=512,
        chunk_overlap=128,
        top_k_retrieval=4,
        temperature=0.1,
        vector_store_type=vector_store_choice
    )

    rag_system = MedicalDocumentRAG(config=config, api_keys=api_keys)

    display(Markdown("\n### 📂 Step 1: Load Document"))

    load_choice = input("Load (U)pload your own document or use (S)ample text? [U/S]: ").strip().lower()

    document_loaded = False

    if load_choice == 'u':
        print("\n--- Local Device File Selection ---")
        print("Please enter the **full local file path** to your document.")
        print("Example: /Users/username/Documents/file.pdf or C:\\Users\\user\\file.docx")
        uploaded_path = input("Enter the path to your document (PDF, TXT, or DOCX): ").strip()

        if uploaded_path:
            try:
                # Assuming the existing load_document method handles reading the file content
                rag_system.load_document(uploaded_path)
                document_loaded = True
                print(f"✓ File loaded from local path: {uploaded_path}")
            except FileNotFoundError:
                print(f"🚨 Error: File not found at path: {uploaded_path}. Please check the path and try again.")
            except Exception as e:
                print(f"🚨 Error loading document: {e}")
        else:
            print("🚨 No file path entered. Document not loaded.")
        
    elif load_choice == 's':
        sample_path = create_sample_document()
        if rag_system.load_document(sample_path, '.txt'):
            document_loaded = True
            print("✓ Sample document loaded.")
    else:
        print("Invalid choice. Exiting.")
        return

    if document_loaded:
        display(Markdown("\n### ⚙️ Step 2: Process and Index"))
        if rag_system.process_and_chunk_document():
            if rag_system.create_embeddings_and_index():
                print("--- System Ready ---")
                stats = rag_system.get_system_stats()
                print(f"LLM Type: {stats['llm_type'].upper()}")
                print(f"Vector Store: {stats['vector_store_type'].upper()}")
                print(f"Total Chunks: {stats['text_chunks']}")
                print("---------------------")

                rag_system.save_index()

                qa_chain = rag_system.create_retrieval_qa_chain()

                display(Markdown("\n###  Step 3: Ask Questions"))
                print("Type 'quit' to exit, 'stats' to see system statistics, 'chain' to switch mode.")

                use_langchain_chain = False
                if qa_chain:
                    use_langchain_chain = input("Start using LangChain QA Chain for generation? (y/n): ").strip().lower() == 'y'

                while True:
                    query = input("\n> Enter your question: ").strip()

                    if query.lower() in ['quit', 'exit', 'q']:
                        break

                    if query.lower() == 'stats':
                        stats = rag_system.get_system_stats()
                        display(Markdown("#### System Stats"))
                        for key, value in stats.items():
                            print(f"  **{key}**: {value}")
                        continue

                    if query.lower() == 'chain':
                        if qa_chain:
                            use_langchain_chain = not use_langchain_chain
                            mode = "LangChain QA Chain" if use_langchain_chain else "Custom RAG"
                            print(f"Switched generation mode to **{mode}**.")
                        else:
                            print("LangChain QA Chain is not available (FAISS not used or LLM failed to init).")
                        continue

                    if not query:
                        continue


                    if use_langchain_chain and qa_chain:
                        display(Markdown(f"\n**Mode: LangChain QA Chain**"))
                        try:
                            result = qa_chain({"query": query})
                            display(Markdown(f"### Answer: {result['result']}"))

                            if 'source_documents' in result:
                                display(Markdown(f"**Sources Used ({len(result['source_documents'])} chunks):**"))
                                for i, doc in enumerate(result['source_documents']):
                                    source_info = doc.metadata.get('source', 'Unknown')
                                    page_info = doc.metadata.get('page', 'Unknown')
                                    print(f"Source {i+1} [File: {source_info}, Page: {page_info}]:")
                                    print(f"  {doc.page_content[:150]}...\n")
                        except Exception as e:
                            print(f"QA Chain error: {e}")
                            print("Falling back to Custom RAG...")
                            result = rag_system.generate_answer(query)
                            use_langchain_chain = False

                    if not (use_langchain_chain and qa_chain):
                        display(Markdown(f"\n**Mode: Custom RAG (LLM: {rag_system.llm_type.upper()})**"))
                        result = rag_system.generate_answer(query)

                        display(Markdown(f"### Answer: {result['answer']}"))

                        if result.get('sources'):
                            display(Markdown(f"**Sources Used ({len(result['sources'])} chunks):**"))
                            for source in result['sources']:
                                similarity_str = f" (Similarity: {source['similarity']:.3f})" if source['similarity'] else ""
                                page_info = source['metadata'].get('page', 'Unknown')
                                print(f"Source {source['source_id']}{similarity_str} [Page: {page_info}]:")
                                print(f"  {source['content']}\n")

                        if 'error' in result:
                            print(f"\n ERROR: {result['error']}")

                display(Markdown("## Thank you for using the Medical RAG system!"))
            else:
                print("Failed to create embeddings and index.")
        else:
            print("Failed to process document.")
    else:
        print("Document loading failed.")


if __name__ == "__main__":

    print("Installing dependencies...")
    os.system("pip install -q langchain langchain-openai langchain-community")
    os.system("pip install -q faiss-cpu sentence-transformers")
    os.system("pip install -q pypdf python-docx tiktoken chromadb")
    os.system("pip install -q openai anthropic cohere transformers torch")
    os.system("pip install -q pinecone-client")

    demonstrate_rag_system()


Installing dependencies...


## 💉 Medical Document RAG System

API Configuration (Optional: Use to enable powerful LLMs/Pinecone)

--- RAG Configuration ---


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000165D9BE44C0>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: e3f10407-cde9-41ff-81d5-7c59a5c7d584)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


Loading embedding model: all-MiniLM-L6-v2


...

✓ Embedding model loaded successfully


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /distilgpt2/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000165D9CA8190>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 7fe35087-3f37-4dc5-819f-3023ae60eab5)')' thrown while requesting HEAD https://huggingface.co/distilgpt2/resolve/main/config.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /distilgpt2/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000165D9CA90F0>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: cbf292c5-9f14-47cc-b3f8-482b6e0451ef)')' thrown while requesting HEAD https://huggingface.co/distilgpt2/resolve/main/config.json
Retrying in 4s [Retry 3/5].
'(MaxRetryError('HTTPS

✓ Using local Hugging Face model (distilgpt2) on device -1



### 📂 Step 1: Load Document

Invalid choice. Exiting.
